#Requirments
%pip install numpy nltk 
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

In [3]:
import numpy as np

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [4]:
#config

#comp

#In the comp function, whenever wordnet.wup_similarity returns none, this value is used instead
uncertApprox = 0.2

#result()

#Threshold for 100% and 50% score from the wordnet.wup_similarity score
normThresh100 = 0.75
normThresh50 = normThresh100/2
#Use the line below to set static values for 50% Threshold
#normThresh50 = 0  

#Normalization values for 100%, 50%, 0%
norm100 = 1
norm50 = 0.5
norm0 = 0

#Number of mismatches to return 
#   If this is greater than the number of sentences, the max number of sentences are returned
mismatchQty = 3

In [5]:
file1 = open('doc1.txt','r')
file2 = open('doc2.txt','r')

a = file1.read()
b = file2.read()

In [6]:
lemmatizer = WordNetLemmatizer()
stop = set(stopwords.words("english"))
stop.remove("not")
stop.remove("no")
posIgnore = ["$","''","(",")",",","--",".",":","CD","FW","LS","NNP","NNPS","NNS","PRP","PRP$","SYM","TO","WDT","WP","WP$","WRB","``"]

In [7]:
def getWnetPos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def simplify(parag):
    '''
    This takes a string 'parag' to tokenize, remove stopwords, lematize the text.
    '''
    stns = sent_tokenize(parag)
    wrds = []
    for stn in stns:
        wrds.append(word_tokenize(stn))
    wrdsFilt = []
    for i in wrds:
        temp = []
        for j in i:
            if j.casefold() not in stop:
                temp.append(j)
        wrdsFilt.append(temp)

    tagged = []
    for words in wrdsFilt:
        tagged.append(nltk.pos_tag(words))
    wrdsLem = []
    for words in tagged:
        temp = []
        for word in words:
            if word[1] in posIgnore:
                temp.append((word[0],word[1]))    
            elif getWnetPos(word[1]) != None:
                temp.append((lemmatizer.lemmatize(word[0],pos=getWnetPos(word[1])),word[1]))
            else:
                temp.append((lemmatizer.lemmatize(word[0]),word[1]))
        wrdsLem.append(temp)
    
    syn = []
    for words in wrdsLem:
        temp = []
        for word,tag in words:
            if word in ['.',',']:
                continue
            temp.append((word,tag))
        syn.append(temp)

    return syn

In [8]:
def nMax(listIn, n):
    '''
    Return n largest elements from ListIn
    '''
    maxN = []
    list1 = listIn.copy()
    for i in range(0, min(n,len(listIn))): 
        max1 = list1[0]
        for e in list1:     
            if e > max1:
                max1 = e  
        
        list1.remove(max1)
        maxN.append(max1)
    return maxN
    
def nMin(listIn, n):
    ''' 
    Return n smallest elements from ListIn
    '''
    minN = []
    list1 = listIn.copy()
    for i in range(0, min(n,len(listIn))): 
        min1 = list1[0]
        for e in list1:     
            if e < min1:
                min1 = e  
        
        list1.remove(min1)
        minN.append(min1)
    return minN

In [9]:
from itertools import product
import nltk
from nltk.corpus import wordnet

def wordSimilarity(wrd1,wrd2):
    '''
    Compares given words using wordnet.sysnet from nltk
    '''
    
    syns1 = wordnet.synsets(wrd1)
    syns2 = wordnet.synsets(wrd2)
    for sn1, sn2 in product(syns1, syns2):
        simil = wordnet.wup_similarity(sn1, sn2)
        if syns1 == syns2:
            return 1.0
        else:
            return simil


def comp(txt1,txt2):
    '''
    Compares 'txt1' and 'txt2' where both are simplified using the simplify function
    '''
    totalScore=[]
    mismatch=[]
    for sA,sB in product(txt1,txt2):
        discard = []
        score = []
        for wA in sA:
            wrdScores = []
            if wA in discard: continue
            for wB in sB:
                if wB in discard: continue
                wrdScores.append(wordSimilarity(wA[0],wB[0])) 
            if wrdScores == []: continue
            wrdScores = [i if i != None else uncertApprox for i in wrdScores] # Replace all uncertain values, ie. None, with a valid approximate value
            score.append(max(wrdScores))
            discard.append(sB[wrdScores.index(max(wrdScores))])
        totalScore.append((sum(nMax(score,min(len(sA),len(sB))))/len(nMax(score,min(len(sA),len(sB)))),(txt1.index(sA),txt2.index(sB))))
        mismatch.append((sum(nMin(score,min(3,len(score))))/len(nMin(score,min(3,len(score)))),(txt1.index(sA),txt2.index(sB))))
    
    scores = []
    temp = []
    if(len(txt1)>len(txt2)):
        for i in range(0,len(txt2)):
            temp.append([])
        for el in totalScore:
            temp[el[1][1]].append(el) 
    else:
        for i in range(0,len(txt1)):
            temp.append([])
        for el in totalScore:
            temp[el[1][0]].append(el) 
    for a in temp:
        if len(a) == 0: continue
        scores.append(max(a))
            
    return scores,nMin(mismatch,mismatchQty)

In [10]:
def result(scores):
    '''
    Normalizes the wup_similarity scores
    '''
    res = []
    for el in scores:
        res.append((norm100 if el[0]>normThresh100 else (norm50 if el[0]>normThresh50 else norm0),el[1]))
    return res

In [11]:
def testRes(scores):
    res = []
    avg = 0
    for el in scores:
        res.append((norm100 if el[0]>normThresh100 else (norm50 if el[0]>normThresh50 else norm0),el[1]))
        avg += norm100 if el[0]>normThresh100 else (norm50 if el[0]>normThresh50 else norm0)
    avg /= len(res)
    return res,avg

In [12]:
scores,mis = comp(simplify(a),simplify(b))
res = result(scores)

score100 = sum([1 if a[0] == norm100 else 0 for a in res])/len([1 if a[0] == norm100 else 0 for a in res])
score50 = sum([1 if a[0] >= norm50 else 0 for a in res])/len([1 if a[0] >= norm50 else 0 for a in res])
score0 = sum([1 if a[0] < norm50 else 0 for a in res])/len([1 if a[0] < norm50 else 0 for a in res])

print("100% Similar: ",score100*100,"%")
print("Atleast 50% Similar: ",score50*100,"%")
print("100% Dissimilar: ",score0*100,"%")
print("Top 3 Mismatches:")
print()


for m in mis:
    print("In First Test:")
    print(sent_tokenize(a)[m[1][0]])
    print("In Second Test:")
    print(sent_tokenize(b)[m[1][1]])
    print()


100% Similar:  40.0 %
Atleast 50% Similar:  100.0 %
100% Dissimilar:  0.0 %
Top 3 Mismatches:

In First Test:
Crucially, Stallman's concept specifically included the idea that no central organization should control editing.
In Second Test:
In 2001, the license for Nupedia was changed to GFDL, and Wales and Sanger launched Wikipedia using the idea and technology of a wiki first discovered/used in 1995 by Ward Cunningham.

In First Test:
Crucially, Stallman's concept specifically included the idea that no central organization should control editing.
In Second Test:
At first, Wikipedia was meant to Nupedia, an online project edited only by experts, by providing more draft articles and ideas for it.

In First Test:
Crucially, Stallman's concept specifically included the idea that no central organization should control editing.
In Second Test:
In practice, Wikipedia quickly ran faster than Nupedia, becoming a worldwide project in many languages and very interesting/intelligent a wide range 